# Homework Starter — Stage 08 EDA

Fill in the marked TODOs. This notebook generates synthetic data so you can focus on the EDA flow. Replace with your dataset when ready.

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from scipy.stats import skew, kurtosis
sns.set(context='talk', style='whitegrid')
np.random.seed(8)
pd.set_option('display.max_columns', 100)

# === Synthetic data generator (adapt or replace with your own data) ===
n = 160
df = pd.DataFrame({
    'date': pd.date_range('2021-02-01', periods=n, freq='D'),
    'region': np.random.choice(['North','South','East','West'], size=n),
    'age': np.random.normal(40, 8, size=n).clip(22, 70).round(1),
    'income': np.random.lognormal(mean=10.6, sigma=0.3, size=n).round(2),
    'transactions': np.random.poisson(lam=3, size=n),
})
base = df['income'] * 0.0015 + df['transactions']*18 + np.random.normal(0, 40, size=n)
df['spend'] = np.maximum(0, base).round(2)

# inject a bit of missingness and outliers
df.loc[np.random.choice(df.index, 5, replace=False), 'income'] = np.nan
df.loc[np.random.choice(df.index, 3, replace=False), 'spend'] = np.nan
df.loc[np.random.choice(df.index, 2, replace=False), 'transactions'] = df['transactions'].max()+12
df.head()

## 1) First look

In [ ]:
df.info(), df.isna().sum()

## 2) Numeric profile

In [ ]:
desc = df[['age','income','transactions','spend']].describe().T
desc['skew'] = [skew(df[c].dropna()) for c in desc.index]
desc['kurtosis'] = [kurtosis(df[c].dropna()) for c in desc.index]
desc

## 3) Distributions (TODO: add at least 3)

In [ ]:
# TODO: histogram + KDE for a key variable
sns.histplot(df['income'], kde=True)
plt.title('Income Distribution')
plt.show()

# TODO: boxplot for outliers
sns.boxplot(x=df['transactions'])
plt.title('Transactions (Outliers)')
plt.show()

In [ ]:
for c in df.select_dtypes(include="number").columns[:3]:
    df[c].plot(kind="hist", bins=30, alpha=0.6)
    plt.title(f"Histogram of {c}")
    plt.show()

    df[[c]].plot(kind="box")
    plt.title(f"Boxplot of {c}")
    plt.show()


## 4) Relationships (TODO: add at least 2)

In [ ]:
sns.scatterplot(data=df, x='income', y='spend', hue='region')
plt.title('Income vs Spend')
plt.show()

sns.scatterplot(data=df, x='age', y='spend')
plt.title('Age vs Spend')
plt.show()

In [ ]:
num_cols = df.select_dtypes(include="number").columns.tolist()
if len(num_cols) >= 2:
    plt.scatter(df[num_cols[0]], df[num_cols[1]], s=12, alpha=0.6)
    plt.xlabel(num_cols[0]); plt.ylabel(num_cols[1])
    plt.title(f"Scatter: {num_cols[0]} vs {num_cols[1]}")
    plt.show()

if len(num_cols) >= 3:
    plt.scatter(df[num_cols[0]], df[num_cols[2]], s=12, alpha=0.6)
    plt.xlabel(num_cols[0]); plt.ylabel(num_cols[2])
    plt.title(f"Scatter: {num_cols[0]} vs {num_cols[2]}")
    plt.show()


## 5) (Optional) Correlation matrix

In [ ]:
corr = df[['age','income','transactions','spend']].corr(numeric_only=True)
sns.heatmap(corr, annot=True, fmt='.2f', cmap='vlag', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()
corr

## 6) Insights & Assumptions (write your commentary)
- TODO: Top 3 insights
- TODO: Assumptions & risks
- TODO: Next steps before modeling (cleaning & features)

### Top 3 Insights
1. Some variables are skewed with visible outliers (seen in hist/boxplots).  
2. Scatterplots suggest linear/positive association between first pairs of numeric features.  
3. At least one variable shows wider spread, which may dominate variance.  

### Assumptions & Risks
- Assumption: numeric features measured consistently.  
- Risk: outliers may distort mean/variance.  
- Risk: possible correlations inflate multicollinearity.  

### Implications for Next Step
- Consider log-transform or winsorize skewed variables.  
- Standardize features before modeling.  
- Examine correlated features to avoid redundancy.  
